# 전처리 시작

## 전처리 기준?
- state 컬럼에서 DELETED, PENDING, REQUESTED는 무슨 뜻인가요? A4. REQUESTED는 고객에게 결제 요청을 보낸 상태입니다. 나머지는 무시하셔도 좋습니다. -> state가 COMPLETED만

- REQUESTED는 패캠이 사용자에게 결제를 요청한것이니 무시

- 데이터 사이언스 -> 데이터사이언스

-  type에서는 PAYMENT와 REFUND만 확인하시면 되고, state에서는 COMPLETED와 REQUESTED만 보시면 됩니다. -> PAYMENT만 보면 안될텐데 이러면..? 흠.. 환불데이터도 봐야하나 (환불한 결제건을 전처리?)

### - customer_id  PAYMENT가 있었는데 -> REFUND가 뜬 경우 (course_id는 같은데) -> 환불한 경우 -> 전처리를 해야 겠다...? 생각했어요

- 쿠폰 금액이 강의 가격보다 더 비싼데 결제를 한 이 데이터들은 어떻게 해석해야할지 여쭤보고 싶습니다. A9. 이 데이터는 무시하셔도 좋습니다. (삭제 하는게 맞음) -> transaction_amount = 0 삭제

-  course_id는 온라인, 오프라인 강의 등 강의의 자체적인 ID로 부여됩니다. 프로모션의 경우에는 따로 ID를 부여하지 않습니다. (강의 ID가 없는건 문제가 되려나?)

- course_id가 null인경우-> 프로모션인데, 프로모션을 분석해야 하나? -> 2168건
- 프로모션 = 올인원 

-  B2B 데이터는 따로 빼서 시간이 남고 여유가 있는 경우 분석 ?

- 스쿨 온라인, 캠프, 스쿨, 온라인 완주반을 합쳐서 하나의 카테고리(2000개 안됨)를 만들고 분석(?) -> 이거는 빼는 걸로 확정 -> 일단은 올인원, RED 로만 분석 가보자 

- 한정 판매ㅡ 프로모션? -> 빼야하는 타당성이 무엇이있을까? -> real data, 진짜 이 과목에 관심있는 사람들에 대한 데이터만 보고싶어서라고 판단.

# 올인원, RED 강의 분석을 통한 강의 트렌드 분석...? 
- 과목 별 선호도 분석
- 서브카테고리 분석 필요
- 쿠폰 분석 필요
- 마케팅 시작 일자 분석
- 
### 유의사항
- transaction 0 인 거 중에 1+1쿠폰이 있었음...
- 결제까지 했다는거는 관심이 있었다.
- 결제건만 보려면 환불을 고려해야함 -> 환불율 분석 가능 (과목별, 강사별,,,?, 강의 퀄리티...)


## 4. "어떻게" 새로운 인사이트를 도출할 계획인가요?
"파레토법칙(Pareto Law)은 전체 결과의 80%가 전체 원인의 20%에서 일어나는 현상을 의미한다. 일례로, 인사행정에 있어서 공무원 20%가 전체성과의 80% 정도를 차지하는 것 등이 이에 해당된다. 대개 80대 20의 법칙, 8대 2의 법칙라고도 한다."

파레토의 법칙을 적용하여 데이터 분석을 진행할 예정. 2021년도 하반기의 매출을 담당할 과목에 대한 인사이트를 얻어내는 목표를 설정

## 5. 프로젝트 진행 계획은 어떻게 되나요?
1. 데이터 전처리
2. 가설 설정
3. 데이터 분석
4. 가설 검정
5. 시각화
6. 발표자료 준비 및 발표 준비

## 6. 진행하면서 어려웠던 점은 무엇인가요?
1. feature가 너무 적다.
2. 데이터 숫자가 적다.
3. column에 대한 설명이 매우 부족하다. (그러므로 인해 그냥 전처리 되는 데이터가 많다)
4. 사용자 관점에 대한 데이터가 부족해서, 분석하기 너무 힘듬
5. 중요한 데이터인데 결측치 존재
6. 데이터의 시간적 구간이 매우 짧음 그리고 최신 데이터도 아님 (2021년 4월~ 9월)



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# row 제한 없애기
# pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
df = pd.read_excel('../파이널프로젝트_RAW_210329_210926.xlsx')

In [3]:
# 구매완료한 강의만 
df = df[df.state=='COMPLETED']
# 데이터 사이언스 == 데이터사이언스 로 바꾸기
df.loc[df.category_title=='데이터 사이언스', 'category_title']= '데이터사이언스'
# payment만 남기고 나머지 전처리
df = df[df.type=='PAYMENT']
# 0원인 강의만 전처리
df = df[df.transaction_amount!=0]
# category_title nan 값 제거 3개
df = df.loc[~(df['category_title'].isnull())]
# course_title, format 'B2B' 전처리
df = df.loc[~(df['course_title'].str.contains('B2B')|(df['format'].str.contains('B2B')))]
# 개발자 테스트용 3건 전처리
df = df[~(df['course_title'].str.contains('TEST|test'))]
# 기간 한정판매 강의 100건 전처리
df = df[~df['course_title'].str.contains('한정판매')]
# 프로모션 강의 2100건 전처리
df = df[~df['course_title'].str.contains('프로모션')]
# 서브카테고리 null 제거
df = df.loc[~(df['subcategory_title'].isnull())]
# format 올인원으로 먼저 파보기
df = df.loc[(df['course_title'].str.contains('올인원')]

In [4]:
df = df.reset_index(drop = True)

In [5]:
df

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
1,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,970669,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
3,970671,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,263000.0,NaN,CARD,NaN,2020-11-27
4,970676,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44184,1364969,669238.0,204286.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 3D 디자인 툴,디자인,올인원,2021-09-25 13:39:26,212000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,232000.0,232000.0,INICIS,CARD,디자인툴,2021-04-23
44185,1364976,384003.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-09-25 13:46:29,149000,NaN,NaN,149000.0,149000.0,KAKAO,CARD,영상 편집,2020-12-02
44186,1364991,670891.0,203935.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 파이썬 웹 개발,프로그래밍,올인원,2021-09-25 14:01:26,159000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,179000.0,179000.0,INICIS,CARD,백엔드 개발,2021-05-31
44187,1364996,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15


In [6]:
df.to_csv("pp_FC.csv", mode='w')

In [7]:
df = pd.read_csv('pp_FC.csv')

In [12]:
df = pd.read_csv('pp_FC.csv', index_col = 0)

In [13]:
df

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04 00:00:00
1,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14 00:00:00
2,970669,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02 00:00:00
3,970671,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,263000.0,NaN,CARD,NaN,2020-11-27 00:00:00
4,970676,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44184,1364969,669238.0,204286.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 3D 디자인 툴,디자인,올인원,2021-09-25 13:39:26,212000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,232000.0,232000.0,INICIS,CARD,디자인툴,2021-04-23 00:00:00
44185,1364976,384003.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-09-25 13:46:29,149000,NaN,NaN,149000.0,149000.0,KAKAO,CARD,영상 편집,2020-12-02 00:00:00
44186,1364991,670891.0,203935.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 파이썬 웹 개발,프로그래밍,올인원,2021-09-25 14:01:26,159000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,179000.0,179000.0,INICIS,CARD,백엔드 개발,2021-05-31 00:00:00
44187,1364996,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15 00:00:00
